In [ ]:
execfile('d:/measuring/analysis/scripts/setup_analysis.py')
#%matplotlib inline

In [ ]:
%matplotlib inline
import os
import numpy as np
import sys
import matplotlib
import matplotlib.pyplot as plt
#plt.rcParams['axes.linewidth'] = 2
label_size = 16
plt.rcParams['xtick.labelsize'] = label_size 
plt.rcParams['ytick.labelsize'] = label_size 
c=2.998e8 #speed of light
import time
filepath='D:/measuring/'
import scipy.signal
os.chdir(filepath)

sys.path.append('H:/My Documents/measuring/')
#%matplotlib inline

from analysis.lib.tools import plot
import analysis.lib.tools.toolbox as tb
from analysis.lib.fitting import fit,common
from analysis.lib.cavity import cavity_tools
import analysis.scripts.cavity.cavity_general_analysis
from analysis.scripts.cavity import analyse_laserscan 

In [ ]:
# NV levels transitions

nvlevels.get_transitions_ExEy(72,80,
                              show_ms0_transitions=True,
                              show_m1_transitions=True,
                              show_p1_transitions=False,
                              show_A_transitions=False,
                              show_FB_A_transitions=False,
                              show_FB_E_transitions=False,
                              show_E_prime_flip_transitions=False)

In [ ]:
#plot transitions 
# Ex and Ey are data from Harry

outdir = r'K:\ns\qt\Diamond\Samples\Cavity samples\2016-09 Harry_unetched\Laserscans'
Ex = np.array([90.91,90.30,110.09,80.33,110.31,93.82,90.44,86.44,80.62,103.21,90.79])
Ey = np.array([83.02,83.20, 91.73, 72.97, 102.46, 75.02, 69.95, 72.95, 66.1, 92.13,80.33])

#Ey_lw1 = np.array([0.315,0.367,0.421,0.295,'NaN',0.668,2.583,'NaN','NaN','NaN','NaN'])

Ey_lw = np.ma.masked_object([0.315,0.367,0.421,0.295,None,0.668,2.583,None,None,None,None], None)
u_Ey_lw = np.ma.masked_object([0.013,0.012,0.102,0.012,None,0.081,0.688, None,None,None,None], None)
#Ey_lw = np.array([0.315,0.367,0.421,0.295,None,0.668,2.583,None,None,None,None])
Ex_lw = np.ma.masked_object([0.332, 0.356, None, None, 1.306, None,None,None,None,2.673,0.974], None)
#u_Ey_lw1 = np.array([0.013,0.012,0.102,0.012,0.081,0.688])
#u_Ey_lw = np.array([0.013,0.012,0.102,0.012,None,0.081,0.688, None,None,None,None])
u_Ex_lw = np.ma.masked_object([0.009, 0.011, None, None, 0.064,None,None,None,None, 0.374, 0.053], None)

strain = Ex - Ey
od = np.array([0.6,1,2,1.8,0.6,0.7,0.1,1.2,1.4,1.3,0.6])
sat_cts = np.array([42, 39,26,44,52,24,27,23,27,27,32])

NVlevels=np.empty((0, 6))
strain_EyE2_lws=[]
strain_Ey=[]
fig,ax=plt.subplots(1, figsize=(12,5))
#fig1,ax1=plt.subplots(1, figsize=(16,5))
#fig2,ax2=plt.subplots(1, figsize=(10,10))
fig3,ax3=plt.subplots(1, figsize=(6,5))
fig4,ax4=plt.subplots(1, figsize=(6,5))
fig5,ax5=plt.subplots(1, figsize=(6,5))
fig6,ax6=plt.subplots(1, figsize=(6,5))


for i in range(len(Ex)):
    T = nvlevels.get_transitions_ExEy(Ey[i],Ex[i],
                              show_ms0_transitions=True,
                              show_m1_transitions=True,
                              show_p1_transitions=False,
                              show_A_transitions=False,
                              show_FB_A_transitions=False,
                              show_FB_E_transitions=False,
                              show_E_prime_flip_transitions=False) 

    NVlevels = np.append(NVlevels,[T], axis=0)
    
E1,E2,Ey,Ex,A1,A2 = NVlevels[:,0],NVlevels[:,1],NVlevels[:,2],NVlevels[:,3],NVlevels[:,4],NVlevels[:,5]

#Translate all tansitions:
E1_t = E1 - Ey-((Ex-Ey)/2)
E2_t = E2 - Ey-((Ex-Ey)/2)
Ex_t = Ex - Ey-((Ex-Ey)/2)
Ey_t = Ey - Ey-((Ex-Ey)/2)
A1_t = A1 - Ey-((Ex-Ey)/2)
A2_t = A2 - Ey-((Ex-Ey)/2)

strain_EyE2 = Ey - E2

# for i in range(len(Ey_lw)):
#     if Ey_lw[i] == 'NaN':
#         continue
#     s_EyE2= Ey[i]-E2[i]  
#     s_Ey = Ey[i]
#     strain_EyE2_lws = np.append(strain_EyE2_lws,[s_EyE2],axis=0)
#     strain_Ey = np.append(strain_Ey,[s_Ey],axis=0)
#     Ey
# print len(strain_EyE2_lws)
# Ey_lw_good = filter(lambda a: a != 'NaN', Ey_lw)
# Ey_lw_good = np.asarray(Ey_lw_good)
# print type(strain_Ey),Ey_lw_good.shape,u_Ey_lw.shape

ax.plot(strain,Ex_t,marker='o',linestyle='', color = 'b',label='Ex')
ax.plot(strain,Ey_t,marker='o',linestyle='', color = 'g',label='Ey')
ax.plot(strain,E1_t,marker='o',linestyle='', color = 'r',label='E1')
ax.plot(strain,E2_t,marker='o',linestyle='', color = 'y',label='E2')
ax.plot(strain,A1_t,marker='o',linestyle='', color = 'black',label='A1')
ax.plot(strain,A2_t,marker='o',linestyle='', color = 'orange',label='A2')
ax.set_xlabel('Lateral strain (GHz)', fontsize = 18)
ax.set_xlim(6,23.5)
ax.set_ylabel('Transition energy(GHz)', fontsize = 18)

#ax.set_title('Strain of all NVs')
ax.legend()
plt.tight_layout()
fig.savefig(os.path.join(outdir,'Strain_allNVs_measured.pdf'))



# ax1.plot(Ex,strain,marker='o',linestyle='', color = 'b')
# ax1.set_xlabel('Position of Ex (GHz wrt 470.4 THz)')
# ax1.set_ylabel('Strain (GHz wrt 470.4 THz)')
# ax1.set_title('Plot of lateral strain vs strain.')

# #ax2.plot(strain_Ey, strain_EyE2_lws, marker='o',linestyle='', color = 'b',label='Ey-E2')
# ax2.plot(strain_Ey, Ey_lw_good, marker='o',linestyle='', color = 'g',label='LW Ey')
# #ax2.plot(strain_Ey, Ey_lw_good, yerr=u_Ey_lw, marker='o', linestyle='', color='g', label='LW Ey')
# ax2.set_xlabel('Position of Ey (GHz wrt 470.4 THz)')
# ax2.set_ylabel('Strain/linewidth (GHz)')
# ax2.set_title('Plot of linewidth Ey and strain between Ey and E2 transitions.')
# ax2.legend()

ax3.plot(od,strain,marker='o',linestyle='', color = 'b' )
ax3.set_xlabel('Optical depth (um)', fontsize = 18)
ax3.set_ylabel('Strain (GHz)', fontsize = 18)
ax3.set_xlim(0,2.5)
plt.tight_layout()
fig3.savefig(os.path.join(outdir,'OD_Strain.eps'))


ax4.plot(strain,Ey_lw, marker='o',linestyle='', color = 'b',label = 'Ey' )
ax4.plot(strain,Ex_lw, marker='o',linestyle='', color = 'g',label = 'Ex' )
ax4.errorbar(strain,Ey_lw,yerr=u_Ey_lw, fmt='.', color = 'b')
ax4.errorbar(strain, Ex_lw, yerr=u_Ex_lw, fmt='.', color='g')
#eb = plt.errorbar(strain, Ex_lw, yerr=u_Ex_lw, fmt='.', color='g', label = 'Ex')
ax4.set_xlabel('Strain (GHz)', fontsize = 18 )
ax4.set_ylabel('Linewidth (GHz)', fontsize = 18)
ax4.set_xlim(5,27)
ax4.legend()
plt.tight_layout()
fig4.savefig(os.path.join(outdir,'Strain_LW.eps'))


ax5.plot(od, sat_cts, marker='o',linestyle='', color = 'b',linewidth = 2 )
ax5.set_xlabel('Optical depth (um)', fontsize = 18 )
ax5.set_ylabel('Saturation counts (Hz)', fontsize = 18)
ax5.set_xlim(0,2.5)
ax5.xaxis.set_tick_params(width=2)
ax5.yaxis.set_tick_params(width=2)
plt.tight_layout()
fig5.savefig(os.path.join(outdir,'OD_sat_cts.eps'))


ax6.plot(od,Ey_lw, marker='o',linestyle='', color = 'b',label = 'Ey', linewidth = 2 )
ax6.plot(od,Ex_lw, marker='o',linestyle='', color = 'g', label = 'Ex' )
#plt.errorbar(strain,Ey_lw,yerr=u_Ey_lw, linestyle="None")
ax6.errorbar(od, Ey_lw, yerr=u_Ey_lw, fmt='.', color='b')
ax6.errorbar(od, Ex_lw, yerr=u_Ex_lw, fmt='.', color='g')
ax6.set_xlabel('Optical depth (um)', fontsize = 18)
ax6.set_ylabel('Linewidth (GHz)', fontsize = 18)
ax6.set_xlim(0,2.5)
ax6.legend()
plt.tight_layout()
fig6.savefig(os.path.join(outdir,'OD_LW.eps'))



plt.show()







In [ ]:
### Fitting laserscans 1 peak at a time

f_min = 85
f_max = 95
g_a= 100
g_A= 300
g_x0 = 90
g_gamma = 0.5
fixed=[]


folder = r'K:\ns\qt\Diamond\Samples\Cavity samples\2016-09 Harry_unetched\Laserscans\Scan1 NV2\183406_laser_scan_Harry_Scan1_NV2_g_0.0_r_1.0'
d = np.loadtxt(tb.get_msmt_fp(folder, ext='dat'), skiprows=15)
if len(d)>2:
    voltage, frequency, counts = d[:,0],d[:,1],d[:,2]
print len(frequency)
fig,ax=plt.subplots(1, figsize=(10,5))
plt.plot(frequency,counts,'-', color ='b')
ax.set_xlabel("Frequency (GHz wrt 470.4 THz)", fontsize = 18)
ax.set_ylabel("Countrate (Hz)", fontsize = 18)
ax.set_xlim(77.5,95)

plt.savefig(os.path.join(folder,'Scan1_NV2_wholefigure_red.eps'), format='eps', dpi=1000)
#plt.savefig(r"K:\ns\qt\Diamond\Samples\Cavity samples\2016-09 Harry_unetched\Laserscans\Scan1 NV2\183406_laser_scan_Harry_Scan1_NV2_g_0.0_r_1.0/Scan1_NV2_wholefigure_red.eps", ext="eps", close=True, verbose=True)
#plt.savefig(r"K:\ns\qt\Diamond\Samples\Cavity samples\2016-09 Harry_unetched\Laserscans\Scan1 NV2\183406_laser_scan_Harry_Scan1_NV2_g_0.0_r_1.0/Scan1_NV2_wholefigure_red.eps", format="eps", dpi=1000)

#plt.savefig('destination_path.eps', format='eps', dpi=1000)

idx_min = np.searchsorted(frequency, f_min, side="left")
idx_max = np.searchsorted(frequency, f_max, side="left")
frequency = frequency[idx_min:idx_max]
counts = counts[idx_min:idx_max]

fig,ax=plt.subplots(1, figsize=(8,5))
    
p0, fitfunc, fitfunc_str = common.fit_lorentz(g_a, g_A, g_x0, g_gamma)
fit_result = fit.fit1d(frequency,counts, None, p0=p0, fitfunc=fitfunc, do_print=True, ret=True, fixed=fixed)

T = fit_result['params_dict']['x0']
gamma = fit_result['params_dict']['gamma']
u_gamma = fit_result['error_dict']['gamma']

print 'T = ', T , 'GHz'
print 'Linewidth T', gamma, 'GHz +/-', u_gamma, 'GHz'


plot.plot_fit1d(fit_result, np.linspace(frequency[0],frequency[-1],len(frequency)),ax=ax, label='Fit',show_guess=True, plot_data=True,color='red', data_linestyle = '-', print_info= False)
ax.set_xlabel("Frequency (GHz wrt 470.4 THz)", fontsize = 14)
ax.set_ylabel("Countrate (Hz)", fontsize = 14)
ax.legend(title='Transition at %.2f GHz, Linewidth = %.2f $\pm$ %.2f MHz'%(T, gamma*1.e3,u_gamma*1.e3))
#plt.savefig(os.path.join(folder,'Scan2_NV1_Ex.png'))


In [ ]:
# plotting optical depth versus linewidth Ey

folder = r'K:\ns\qt\Diamond\Samples\Cavity samples\2016-09 Harry_unetched\Laserscans'
fig,ax=plt.subplots(1, figsize=(6,5))
od = [0.6, 1, 2, 1.8]
Ey = [315, 367, 421, 295]
yerr = [13, 12, 102, 12]

eb = plt.errorbar(od, Ey, yerr=yerr, fmt='.', color='b')
ax.set_xlabel("Optical depth (um)", fontsize = 18)
ax.set_ylabel("Linewidth Ey (MHz)", fontsize = 18)
ax.set_xlim(0,2.5)
#plt.savefig(os.path.join(folder,'OpticalDepth_vs_Eylinewidth.png'))


strain = [6.99,7.1,18.36,7.36,7.84,18.8,20.49,13.49,14.52]
avg_strain = np.mean(strain)
print avg_strain

In [ ]:
# Plotting 2D sweep

import analysis.scripts.cavity.spectrometer_analysis as sa
import analysis.scripts.cavity.spectrometer_analysis_2Dsweep as spectrometer_analysis_2Dsweep
reload(sa)
reload(spectrometer_analysis_2Dsweep)
data_dir = 'D:\measuring\data/20161025/20160330 Sweep2Dplot'

sa_2D = spectrometer_analysis_2Dsweep.spectrometer_2D_analysis(data_dir)#,V_min=0,V_max=10,laser_wavelength=636.6e-9)
frequencies,filenumbers,intensities = sa_2D.get_data()#(min_frq=470,max_frq=473)
print frequencies[0]
sa_2D.plot_data(aspect =0.1,ret_ax=True, vmax=2500,vmin=0,save_fig=True)



In [ ]:
### Plotting two graphs in one figure same axes

import matplotlib.pyplot as plt
%matplotlib inline

import analysis.scripts.cavity.spectrometer_analysis as sa
import analysis.scripts.cavity.spectrometer_analysis_2Dsweep as spectrometer_analysis_2Dsweep

data_dir = 'D:\measuring\data/20161004/Sophie_ZPL_spectrometer/20160920 ZPL Sophie Area 5'
data_dir2 = 'D:\measuring\data/20161004/Harry_ZPL_spectrometer'

sa_2D = spectrometer_analysis_2Dsweep.spectrometer_2D_analysis(data_dir)#,V_min=0,V_max=10,laser_wavelength=636.6e-9)
frequencies,filenumbers,intensities = sa_2D.get_data()#(min_frq=470,max_frq=473)
print frequencies[0]

intensity = intensities[:,0]
intensity = intensity - np.mean(intensity)
#intensity = intensity/600

sa_2D_2 = spectrometer_analysis_2Dsweep.spectrometer_2D_analysis(data_dir2)#,V_min=0,V_max=10,laser_wavelength=636.6e-9)
frequencies2,filenumbers2,intensities2 = sa_2D_2.get_data()#(min_frq=470,max_frq=473)

intensity2 = intensities2[:,0]

f_min = 470.5
f_max = 480
idx_min = np.searchsorted(frequencies, f_min, side="left")
print idx_min
idx_max = np.searchsorted(frequencies, f_max, side="left")
print idx_max
frequency = frequencies[idx_min:idx_max]
intensity = intensity[idx_min:idx_max]

idx_min2 = np.searchsorted(frequencies2, f_min, side="left")
idx_max2 = np.searchsorted(frequencies2, f_max, side="left")
frequency2 = frequencies2[idx_min2:idx_max2]
intensity2 = intensity2[idx_min2:idx_max2]
print frequency2

fig,ax=plt.subplots(1, figsize=(5,5))
plt.plot(frequency,intensity, '-', label='d = 3 um', linewidth = 2)
plt.plot(frequency2, intensity2, '-', label='d = 27 um', linewidth = 2)

plt.xlabel('Frequency (THz)', fontsize = 18)
plt.ylabel('Intensity (a.u.)', fontsize = 18)
ax.set_ylim(0,5000)
ax.set_xlim(470.5,472.4)
#ax.set_yscale('log')
plt.legend()
label_size = 16
plt.rcParams['xtick.labelsize'] = label_size 
#plt.title('Sophie"s ZPL is translated down 600, for clarity')
plt.tight_layout()
plt.draw()
#plt.show()
folder = r'K:\ns\qt\Diamond\Samples\Cavity samples\2016-09 Harry_unetched\Spectrometer_Scan4NV1_ZPL'
plt.savefig(os.path.join(folder,'ZPL_Sophie_Spectrometer.pdf'))



In [ ]:
### Plotting and fitting spectrometer data

import analysis.scripts.cavity.spectrometer_analysis as sa
import analysis.scripts.cavity.spectrometer_analysis_2Dsweep as spectrometer_analysis_2Dsweep
data_dir = 'D:\measuring\data/20161025\single_bare_meas'
min_frq = 400
max_frq = 500
f_min = 0
f_max = 1000
# g_a= 0
# g_A= 100
# g_x0 = 471.7
# g_gamma = 0.5
# fixed=[]

sa_2D = spectrometer_analysis_2Dsweep.spectrometer_2D_analysis(data_dir)#,V_min=0,V_max=10,laser_wavelength=636.6e-9)
frequencies,filenumbers,intensities = sa_2D.get_data()#min_frq=470,max_frq=473)
reload(spectrometer_analysis_2Dsweep)
intensity = intensities[:,0]
print type(intensity)
intensity = intensity
fig,ax=plt.subplots(1, figsize=(6,5))
plt.plot(frequencies,intensity,'-', linewidth =2 )
ax.set_xlabel("Frequency (THz)", fontsize = 18)
ax.set_ylabel("Intensity(a.u.)", fontsize = 18)
ax.xaxis.set_tick_params(width=2)
ax.yaxis.set_tick_params(width=2)
ax.set_xlim(min(frequencies),max(frequencies))

plt.tight_layout()

plt.savefig(os.path.join(data_dir,'single_bare_sweep.eps'))

# idx_min = np.searchsorted(frequencies, f_min, side="left")
# idx_max = np.searchsorted(frequencies, f_max, side="left")
# frequency = frequencies[idx_min:idx_max]
# intensity = intensity[idx_min:idx_max]
# print max(frequency)
# print len(frequency)


# fig,ax=plt.subplots(1, figsize=(8,5))
    
# p0, fitfunc, fitfunc_str = common.fit_lorentz(g_a, g_A, g_x0, g_gamma)
# fit_result = fit.fit1d(frequency,intensity, None, p0=p0, fitfunc=fitfunc, do_print=True, ret=True, fixed=fixed)

# T = fit_result['params_dict']['x0']
# gamma = fit_result['params_dict']['gamma']
# u_gamma = fit_result['error_dict']['gamma']

# print 'T = ', T , 'GHz'
# print 'Linewidth T', gamma, 'GHz +/-', u_gamma, 'GHz'


# plot.plot_fit1d(fit_result, np.linspace(frequency[0],frequency[-1],len(frequency)),ax=ax, label='Fit',show_guess=True, plot_data=True,color='red', data_linestyle = '-', print_info= False)
# ax.set_xlabel("Frequency (THz)", fontsize = 14)
# ax.set_ylabel("Intensity (a.u.)", fontsize = 14)
# ax.set_yscale('log')
# ax.legend(title='ZPL at %.2f THz, Linewidth = %.2f $\pm$ %.2f GHz'%(T, gamma*1.e3,u_gamma*1.e3))
# plt.savefig(os.path.join(data_dir,'fit_Sophie_scan5.png'))

In [ ]:
# plotting microwave power versus linewidth Ex

# folder = r'K:\ns\qt\Diamond\Samples\Cavity samples\2016-09 Harry_unetched\Laserscans\Plots from sweeps\Scan4NV1_microwave dependence'
# filename = r'K:\ns\qt\Diamond\Samples\Cavity samples\2016-09 Harry_unetched\Laserscans\Plots from sweeps\Scan4NV1_microwave dependence/mw_dependence_Scan4NV1.txt'
# #d = np.loadtxt(tb.get_msmt_fp(folder, ext='txt'), skiprows=15)
# d = np.loadtxt(filename, skiprows=15)
# if len(d)>2:
#     mw_power, lw, error_lw, red_power = d[:,0],d[:,1],d[:,2],d[:,3]

# fig,ax=plt.subplots(1, figsize=(8,5))
# eb = plt.errorbar(mw_power, lw, yerr=error_lw, fmt='.', color='b')
# ax.set_xlabel("Microwave power (dBm)", fontsize = 14)
# ax.set_ylabel("Linewidth Ex (MHz)", fontsize = 14)
# ax.set_xlim(5,30)
# plt.savefig(os.path.join(folder,'Microwavepower_vs_Exlinewidth_Scan4NV1.png'))


folder = r'K:\ns\qt\Diamond\Samples\Cavity samples\2016-09 Harry_unetched\Laserscans'
fig,ax=plt.subplots(1, figsize=(6,5))
mw = [10, 10,12,15,15,20,26]
Ex = [975, 953,975,912,993,1284,2774]
yerr = [53,50,67,63,64,91,108]

eb = plt.errorbar(mw, Ex, yerr=yerr, fmt='o', color='b')
ax.set_xlabel("Microwave power (dBm)", fontsize = 16 )
ax.set_ylabel("Linewidth Ex (MHz)", fontsize = 16)
ax.set_xlim(5,30)
#plt.xticks(np.arange(min(pw), max(pw)+1, 10.0))
#plt.yticks(np.arange(1000, 1900, 200.0))

plt.tight_layout()
plt.savefig(os.path.join(folder,'Microwavepower_vs_Exlinewidth_Scan5NV4.eps'))



In [ ]:
### Plotting multiple laserscans and sum/average/etc. 

# outdir = where the plots are saved 
#outdir=r'K:\ns\qt\Diamond\Samples\Cavity samples\2016-09 Harry_unetched\Laserscans\Plots from sweeps\Scan1NV2_Sweep_no_MW'
outdir=r'K:\ns\qt\Diamond\Samples\Cavity samples\2016-09 Harry_unetched\Laserscans\Plots from sweeps\Scan4NV1_Sweep_red_green'
older_than='20161011113300'#'20161012183022'#20161011113300'#20161012182200'#'20161005130700'
newer_than ='20161011103600'#'20161012174152'#20161011103600'#20161012174000'#'20161005121700'

# making the figures and axes
fig,ax=plt.subplots(1, figsize=(5,5))
fig2,ax2=plt.subplots(1, figsize=(8,8))
fig3,ax3 = plt.subplots(1, figsize=(5,5))
fig4,ax4=plt.subplots(1, figsize=(8,8))

# making of the bin arrays and histogram arrays
bins = np.arange(76,83,.05)
n_xticks = (max(bins)-min(bins))+1 
bins_shifted = bins-(min(bins)+((max(bins)-min(bins))/2))
hist = np.zeros((len(bins)-1))
hist_shifted = np.zeros((len(bins)-1))
bin_norm = np.zeros(len(bins)-1)
hist_2D=np.empty((0, len(bins)-1))
ii = 0

#fitting parameters for figure 1 
g_a = -5
g_A = 150
g_x0 = 80
g_gamma = 1.1
g_a1 = 150
g_A1 = 150 
g_x01 = 80
g_gamma1 = 1.1
fixed = []


# getting the data, 
while 1:
    #ret = tb.latest_data(contains='LaserScanGreenRed', older_than = older_than,newer_than=newer_than,return_timestamp=True, raise_exc=False)
    ret = tb.latest_data(contains='laser_scan', older_than = older_than,newer_than=newer_than,return_timestamp=True, raise_exc=False)
    if not ret:
        break
    timestamp,folder = ret
    older_than=timestamp
    # added ionised to folder that has ionised data, to get it out - horrible method I know
    if 'ionised' in folder:
        print folder
        continue        
    
    d = np.loadtxt(tb.get_msmt_fp(folder, ext='dat'), skiprows=15) 
    if len(d)>2:
        voltage, frequency, counts = d[:,0],d[:,1],d[:,2] 

    if len(counts)>100:
        # 'normalizing' the counts
        cts = counts.astype(np.float)/np.average(counts) -1    
        # fitting the data and shifting the bins to zero to get the intrinsic linewidth
        p0, fitfunc, fitfunc_str = common.fit_lorentz(g_a, g_A, g_x0, g_gamma)
        fit_result = fit.fit1d(frequency,cts, None, p0=p0, fitfunc=fitfunc, do_print=False, ret=True, fixed=fixed)
        x0 = fit_result['params_dict']['x0']
        frq_shifted = frequency-x0
        # building up the histograms of both shifted and normal data
        _h,_b = np.histogram(frequency, weights=cts, bins=bins)
        _h_shifted,_b_shifted = np.histogram(frq_shifted, weights=cts, bins = bins_shifted)
        # appending all data in 2D array for 2D plot
        hist_2D = np.append(hist_2D, [_h], axis=0) 
        # appending all counts to build up the histogram of normal and shifted data
        hist+= _h
        hist_shifted+=_h_shifted
        # plot all data in one figure  
        ax.plot(frequency,cts, alpha=1, linestyle='dotted', color = 'b')
        #ax3.plot(frq_shifted,cts, alpha=1, linestyle='dotted', color = 'b' )
        ii+=1
        
#print frq_shifted[0], frequency[0], cts[0]
hist_shifted = hist_shifted + 150
print hist_shifted
print hist

# plotting figure 1 - all normal data plotted in one figure 
ax.set_xlabel('Frequency (GHz wrt 470.4 THz)', fontweight = 'bold', fontsize = 14)
ax.set_ylabel('Deviation from average countrates (Hz)', fontweight = 'bold', fontsize = 14)
ax.set_xlim(min(bins),max(bins))
#ax.set_title(timestamp+'_'+os.path.split(folder)[-1])

# plotting figure 2 histogram of normal data 
ax2.plot(bins[1:],hist,color='b',linestyle = '',marker = 'o', label = 'avg')
ax2.set_xlim(min(bins),max(bins))
ax2.set_xlabel('Frequency (GHz wrt 470.4 THz)', fontsize = 18)
ax2.set_ylabel('Averaged countrates (Hz)', fontsize = 18)
plt.tight_layout()
#ax2.set_title(timestamp+'_'+os.path.split(folder)[-1])
#ax2.legend()


# plotting figure 3 - plotting shifted data 
ax2.plot(bins_shifted[1:]+x0,hist_shifted, linestyle='',marker = 'o',color='g', label = 'avg + \n centered') 
#ax2.set_xlim(min(bins_shifted),max(bins_shifted))
#ax2.set_xlabel('Detuning (GHz)', fontweight = 'bold', fontsize = 14)
#ax2.set_ylabel('Deviation from average countrates (Hz)', fontweight = 'bold', fontsize = 14)
#ax3.set_title(timestamp+'_'+os.path.split(folder)[-1])


# fitting and plotting figure 2 of the normal data 
p0, fitfunc, fitfunc_str = common.fit_lorentz(g_a, g_A, g_x0, g_gamma)
fit_result = fit.fit1d(bins[1:],hist, None, p0=p0, fitfunc=fitfunc, do_print=True, ret=True, fixed=fixed)
x0 = fit_result['params_dict']['x0'] 
gamma = fit_result['params_dict']['gamma']
u_gamma = fit_result['error_dict']['gamma']
plot.plot_fit1d(fit_result, np.linspace(bins[1:][0],bins[1:][-1],len(bins)-1),ax=ax2, label='Fit',show_guess=True, plot_data=False,color='r', data_linestyle = '-', print_info= False)

ax2.legend()

# fitting and plotting of the shifted data figure 3
p01, fitfunc1, fitfunc_str1 = common.fit_lorentz(g_a1, g_A1, g_x01, g_gamma1)
fit_result1 = fit.fit1d(bins_shifted[1:]+x0,hist_shifted, None, p0=p01, fitfunc=fitfunc1, do_print=True, ret=True, fixed=fixed1)
x01 = fit_result1['params_dict']['x0'] 
gamma1 = fit_result1['params_dict']['gamma']
u_gamma1 = fit_result1['error_dict']['gamma']
plot.plot_fit1d(fit_result1, np.linspace(bins_shifted[1:][0]+x0,bins_shifted[1:][-1]+x0,len(bins)-1),ax=ax2, label='Fit',show_guess=True, plot_data=False,color='r', data_linestyle = '-', print_info= False)
plot.plot_fit1d(fit_result, np.linspace(bins[1:][0],bins[1:][-1],len(bins)-1),ax=ax2, label='Fit',show_guess=True, plot_data=False,color='r', data_linestyle = '-', print_info= False)
plt.tight_layout()


# plotting figure 4 - 2D plot 
#ax4.set_title(timestamp+'_'+os.path.split(folder)[-1])
ax4.set_xlabel('Frequency (GHz wrt 470.4 THz)', fontsize = 18)
ax4.set_ylabel('Scan number', fontsize = 18)
ax4.set_ylim(0,ii)
#ax4_set_xlim(min(bins),max(bins))
xticklabels = np.linspace(min(bins),max(bins),n_xticks)
xticklabels_round=[]
for j in xticklabels:
    round_ = int(round(j))
    xticklabels_round = np.append(xticklabels_round,round_)
ax4.set_xticklabels(xticklabels_round.astype(int))
ax4.pcolormesh(hist_2D, cmap='Greys_r', alpha = 1)
plt.tight_layout()
# getting the linewidths of both datasets and deduce spectral diffusion


print 'Linewidth of normal data = ', round(gamma*1.e3),'+/-',round(u_gamma*1.e3), 'MHz'
print 'Linewidth of shifted data = ', round(gamma1*1.e3),'+/-',round(u_gamma1*1.e3), 'MHz'
if abs(gamma - gamma1) > u_gamma1:
        print 'Estimate of spectral diffusion = ', round(gamma*1.e3 - gamma1*1.e3), 'MHz'
else:
    print 'errorbars larger than difference between linewidth, no estimate of spectral diffusion possible. '
plt.show()

# saving the data in outdir
# fig.savefig(os.path.join(outdir, timestamp+'_sweep_LaserScan_greenred_Scan4NV1_allplots.pdf'))
fig2.savefig(os.path.join(outdir, timestamp+'_sweep_LaserScan_greenred_Scan4NV1_averaged_shifted_together.png'))
# fig3.savefig(os.path.join(outdir, timestamp+'_sweep_LaserScan_greenred_Scan4NV1_averaged_shifted.pdf')) 
fig4.savefig(os.path.join(outdir, 'sweep_LaserScan_greenred_Scan4NV1_2Dplot.png'))
# fig.savefig(os.path.join(outdir, timestamp+'_sweep_LaserScan_greenred_Scan4NV1_allplots.png'))
#fig2.savefig(os.path.join(outdir, timestamp+'_sweep_LaserScan_greenred_Scan4NV1_averaged_shifted_together.png'))
# fig3.savefig(os.path.join(outdir, timestamp+'_sweep_LaserScan_greenred_Scan4NV1_averaged_shifted.png')) 
#fig4.savefig(os.path.join(outdir, 'sweep_LaserScan_greenred_Scan4NV1_2Dplot.png'))




In [ ]:
#Calibration of the red laser 
t = np.array([637.009, 637.04,637.06,637.153,637.289])
d = np.array([250, 232.1, 217.1, 153.4,49.4])

x,y=1/t,d
fig,ax=plt.subplots(1, figsize=(16,5))

f = common.fit_line
args=[0,100]
fitres = fit.fit1d(x, y, f,*args, fixed = [],
                   do_print = True, ret = True, maxfev=100)
p1 = fitres['params_dict']
ax.plot(x,y, 'o')
plot_pts=100
x_p=np.linspace(min(x),max(x),plot_pts)
if fitres['success']:
    f_p = fitres['fitfunc'](x_p)
    ax.plot(x_p,f_p)
f_g = f(*args)[1](x_p)
ax.plot(x_p,f_g)

print fitres['fitfunc'](1/637.16)
print fitres['fitfunc'](1/636.526)

In [ ]:
print 199*17

In [ ]:
print int(2.3)

In [ ]:
#Fit ESR
execfile(r'D:\measuring\analysis\scripts\setup_analysis.py')
from analysis.lib.fitting import fit, common,esr
import inspect
%matplotlib inline
older_than = '20161011114510'
newer_than = '20161011114500'
nr_of_scans = 1
fig,ax=plt.subplots(1, figsize=(6,5))
folder1 = r'K:\ns\qt\Diamond\Samples\Cavity samples\2016-09 Harry_unetched\Laserscans'


f_min = 2.8e9
f_max = 2.95e9

#fitting parameters for figure 1 
g_a = 5500
g_A = 200 
g_x0 = 2.878e9
g_sigma = 1e8
fixed =[]

for i in range(nr_of_scans):
    #ret = tb.latest_data(contains='LaserScanGreenRed', older_than = older_than,newer_than=newer_than,return_timestamp=True, raise_exc=False)
    ret = tb.latest_data(older_than = older_than,newer_than = newer_than, return_timestamp=True, raise_exc=False)
    print ret
    if not ret:
        break
    timestamp,folder = ret
    older_than=timestamp
    
    d = np.loadtxt(tb.get_msmt_fp(folder, ext='dat'), skiprows=15)
    frequency, counts = d[:,0],d[:,1] 
    idx_min = np.searchsorted(frequency, f_min, side="left")
    idx_max = np.searchsorted(frequency, f_max, side="left")
    frequency = frequency[idx_min:idx_max]
    frequency = frequency*1.e-9
    counts = counts[idx_min:idx_max]
    p0, fitfunc, fitfunc_str = esr.fit_ESR_gauss(g_a, g_A, g_sigma, g_x0)
    fit_result = fit.fit1d(frequency,counts, None, p0=p0, fitfunc=fitfunc, do_print=True, ret=True, fixed=fixed)
    #plot.plot_fit1d(fit_result, np.linspace(frequency[0],frequency[-1],len(frequency)),ax=ax, label='Fit',show_guess=True, plot_data=True,color='g', data_linestyle = '-', print_info= False)
    ax.plot(frequency,counts, alpha=1, linestyle='-', color = 'b', linewidth=2)
    ax.set_xlabel('Frequency (GHz)', fontsize = 18)
    ax.set_ylabel('Counts', fontsize = 18)
    ax.set_xticks(np.arange(2.80, 2.96, 0.04))
    plt.tight_layout()
    #ax.set_title(timestamp+'_'+os.path.split(folder)[-1])
    fig.savefig(os.path.join(folder1, timestamp+'_ESR_Scan4NV1.png'))


In [ ]:
#Fit Saturation curve
execfile(r'D:\measuring\analysis\scripts\setup_analysis.py')
from analysis.lib.fitting import fit, common,esr
import inspect
#from pylab import *
%matplotlib inline
older_than = '20161010191000'
newer_than = '20161010190800'
nr_of_scans = 1
fig,ax=plt.subplots(1, figsize=(6,5)) 
folder1 = r'K:\ns\qt\Diamond\Samples\Cavity samples\2016-09 Harry_unetched\Laserscans'


#fitting parameters for figure 1 
g_A = 30000
g_xsat = 260e-6
 
max_power = 300e-6

fixed =[]

for i in range(nr_of_scans):
    #ret = tb.latest_data(contains='LaserScanGreenRed', older_than = older_than,newer_than=newer_than,return_timestamp=True, raise_exc=False)
    ret = tb.latest_data(contains='Saturation', older_than = older_than,newer_than = newer_than, return_timestamp=True, raise_exc=False)
    if not ret:
        break
    timestamp,folder = ret
    older_than=timestamp
    
    d = np.loadtxt(tb.get_msmt_fp(folder, ext='dat'), skiprows=4)
    power, counts, counts_minBG = d[:,0],d[:,1],d[:,2] 


    p0, fitfunc, fitfunc_str = common.fit_saturation(g_A, g_xsat)
    fit_result = fit.fit1d(power,counts_minBG, None, p0=p0, fitfunc=fitfunc, do_print=True, ret=True, fixed=fixed)
    plot.plot_fit1d(fit_result, np.linspace(power[0],power[-1],len(power)),ax=ax, label='Fit',labelsize = 18, show_guess=True, plot_data=False,color='r', data_linestyle = '-', print_info= False)
    ax.plot(power,counts_minBG, alpha=1, linestyle='',marker = 'o', color = 'b', linewidth=2)
    ax.set_xlabel('Power (uW)',fontsize = 18)
    ax.set_ylabel('Countrate (Hz)',fontsize = 18)
    #rc('axes', linewidth=2)
#     ax.text(.5,.9,'Sat. cts: 29.4 kcts, sat. pwr: 260 uW',
#     horizontalalignment='right',
#     transform=ax.transAxes)
    #ax.set_title('Sat. cts: 29.4 kcts\nSat. pwr: 260 uW', position=(0.3, 0.8), fontweight = 'bold', fontsize = 14)
    #ax.set_title('Sat. cts: {:d}, sat. pwr: {:.2f} uW'.format(int(fit_res['params_dict']['A']),fit_res['params_dict']['xsat']))
    #ax.legend()
    plt.legend(bbox_to_anchor=(0.95, 0.8),
           bbox_transform=plt.gcf().transFigure)
    ax.text(.05, .85, 'Sat. cts: 29.4 kcts\nSat. pwr: 260 uW', color='black', fontsize = 18 , 
        bbox=dict(facecolor='none', edgecolor='black'), horizontalalignment='left', transform=ax.transAxes)
    plt.tight_layout()
    #ax.set_title(timestamp+'_'+os.path.split(folder)[-1])
    fig.savefig(os.path.join(folder1, timestamp+'_Saturationcurve_Scan4NV1.png'))

In [ ]:
# Plotting red power broadening

folder = r'K:\ns\qt\Diamond\Samples\Cavity samples\2016-09 Harry_unetched\Laserscans'
fig,ax=plt.subplots(1, figsize=(6,5))
pw = [10, 20, 40]
Ex = [1168, 1442, 1689]
yerr = [158, 97, 82]

eb = plt.errorbar(pw, Ex, yerr=yerr, fmt='o', color='b')
ax.set_xlabel("Power (nW)", fontsize = 18)
ax.set_ylabel("Linewidth Ex (MHz)", fontsize = 18)
ax.set_xlim(5,45)
plt.xticks(np.arange(min(pw), max(pw)+1, 10.0))
plt.yticks(np.arange(1000, 1900, 200.0))

plt.tight_layout()
plt.savefig(os.path.join(folder,'Power_broadening_red.eps'))

In [ ]:
#Plotting and fitting scope data


%matplotlib inline
import analysis.scripts.cavity.spectrometer_analysis as sa
import analysis.scripts.cavity.spectrometer_analysis_2Dsweep as sa_2D
import analysis.scripts.cavity.oscilloscope_analysis_linewidth as oscillo_ana
import pandas as pd
indir='D:\measuring/data/20161025/20160419 Linewidth'#'D:\measuring\data/20160430\OFF_diamond'
filename='FRI021_FSR_50nm_RFON_20DB'
EOM_freq=6 #GHz

#Selecting the correct peak
x_min = 3500 #Left most point of the x (in ms)
x_max = 7500 #Right most point in x (in ms)

oa = oscillo_ana.oscilloscope_analysis(indir=indir,filename=filename)
data = pd.read_csv(os.path.join(indir,filename+'.csv'), skiprows=1, names = ["2","3","None"],usecols=[0,1]) #creating a dataframe in pandas and importing the data
x,y = data['2'], data['3']
x = np.asarray(x)*1.e9
y = np.asarray(y)
print len(x)

x = x[x_min:x_max]
y = y[x_min:x_max]

n_xticks = 10

#Fit parameters 
g_a1 = 0.04 #Offset in the y-axis (usually remains at 0)
g_A1 = 500 # Area of the large peaks ((High time-Low time)*Intensity)
g_gamma1 = 1000 #Estimate of linewidth of the center peak (Width at half max (in ms))
#g_gamma2 = 200
g_dx = 3500 #Difference in ms between the center peak and the side peaks
g_A2 =  500 #Area of the side peak on the left
g_x01 = 65 #Automatically finds the maximum of the middle peak. You have to place the middle peak in the center of the image for this to get a good estimate. Alternatively you can just input g_x01 manually if you have multiple peaks. 
#g_x01=22250
fixed =[]

p0, fitfunc, fitfunc_str = common.fit_3lorentz_symmetric(g_a1, g_A1, g_x01, g_gamma1, g_dx, g_A2)

fit_result = fit.fit1d(x,y, None, p0=p0, fitfunc=fitfunc, do_print=True, ret=True, fixed=fixed)
dx = fit_result['params_dict']['dx']
x01 = fit_result['params_dict']['x01']
gamma1 = fit_result['params_dict']['gamma1']
u_gamma1 = fit_result['error_dict']['gamma1']
scaling = EOM_freq/dx #scale the known EOM freq with the separation here.
linewidth = gamma1*scaling #scale the linewidth to get linewidht in frequency
u_linewidth = u_gamma1*scaling
linewidth_string = 'gamma = '+str(round(linewidth,2))+'+-'+str(round(u_linewidth,3))+'GHz'
print linewidth_string

#plot data 
fig,ax = plt.subplots(figsize=(6,5))
ax.plot(x,y, linestyle = ':', marker = '', color = 'b', alpha = 1)

plot.plot_fit1d(fit_result, np.linspace(x[0],x[-1],len(x)),ax=ax, label='Fit',show_guess=True, plot_data=False,color='red', data_linestyle = '-', print_info= False)
ax.set_ylabel("Intensity (a.u.)", fontsize = 18)
ax.set_xlabel("Frequency (GHz)", fontsize = 18)
ax.set_xlim(x[0],x[-1])
xticks = np.linspace(ax.get_xlim()[0],ax.get_xlim()[-1],n_xticks)
#rescaling for x-axis in GHz
X_min_freq = ax.get_xlim()[0]*scaling
X_max_freq = ax.get_xlim()[-1]*scaling


xticklabels = np.linspace(X_min_freq,X_max_freq,n_xticks)

xticklabels_round=[]
for j in xticklabels:
    round_ = round(j,0)
    xticklabels_round = np.append(xticklabels_round,round_)

ax.set_xticks(xticks)
plt.tight_layout()
ax.set_xticklabels(xticklabels_round.astype(int))
#ax.set_title(indir+'/'+filename+'\n'+linewidth_string)
plt.savefig(os.path.join(indir,filename+'.pdf'))
plt.show()

In [ ]:
# Purcell enhancement
from math import sqrt
LW_cav = 5.e9 
LW_ZPL = 1.5e9
t_d = 6.e-6
t_a = 2.e-6
ROC = 20.e-6
lamd = 470.4e12
lamd_wl = 637.e-9
gamma_PSB = 0.32e12
gamma_ZPL = 10.e9
w_c = 470.4e12
e_0 = 8.854e-12
h_bar = 1.0545e-34

Q_cav = lamd/LW_cav
Q_em = lamd/LW_ZPL

Q_eff = 1/((1/Q_cav)+(1/Q_em))
print Q_eff

L = (t_d *2.4) + t_a

w0 = (float(lamd_wl**(0.5))/np.pi)*(L*(ROC - L))**(float(1)/4)
print w0

V0 = float((np.pi*(w0**2)*L))/4
print V0

Omega_0 = (float(w_c)/(2*e_0*h_bar*V0))**(float(1)/2)
print 'Omega_0 =', Omega_0

e=  (w_c)**(float(1)/2)
print e
d= (2*e_0*h_bar*V0)**(float(1)/2)
print d
W = float(e)/float(d)
print W
F = (3 * Q_eff * (lamd_wl)**3)/(4*np.pi**2*V0)
print 'F = ',F

B_cav = ((F+1)*gamma_ZPL)/(((F+1)*gamma_ZPL)+gamma_PSB)
print B_cav

speedup = (0.87/0.03)**2
print speedup

print 3600/speedup

In [ ]:
c = 3.e8
n = 2.4

filepath = 'D:\measuring\data/20161025/linewidth_data.txt'
d = np.loadtxt(filepath)
lw,u_lw = d[:,0]*1.e9, d[:,1]*1.e9

filepath2 = 'D:\measuring\data/20161025/fsr_data.txt'
t = np.loadtxt(filepath2)
FSR, u_L = t[:,0], t[:,2]
L = c/(FSR*2)

print L, u_L

fig,ax = plt.subplots(figsize=(8,5))
eb = plt.errorbar(L, lw, yerr=u_lw,xerr=u_L, fmt='o', color='b')
ax.set_xlabel("Cavity length (um)", fontsize = 18)
ax.set_ylabel("Linewidth (GHz)", fontsize = 18)
#ax.set_xlim(4,15)

g_a = 0.1e9
g_b = (c)/(2*40000)

p0, fitfunc, fitfunc_str = common.fit_inverse(g_a, g_b)
fit_result = fit.fit1d(L,lw, None, p0=p0, fitfunc=fitfunc, do_print=True, ret=True, fixed=[])
print p0

plot.plot_fit1d(fit_result, np.linspace(min(L),max(L),10*len(L)),ax=ax, label='Fit',show_guess=True, plot_data=False,color='red', data_linestyle = 'o', print_info= False)
b_fit=fit_result['params_dict']['b']
Finesse = c/(2*b_fit)
print Finesse
#ax.set_title(str(Finesse))
xticklabels = np.linspace(round(min(L)*1.e6,0), round(max(L)*1.e6,0), 6)
ax.set_xticklabels(xticklabels.astype(int))
#yticklabels = np.linspace(min(lw)*1.e-9, max(lw)*1.e-9, 6)
#ax.set_yticklabels(yticklabels)
plt.legend(bbox_to_anchor=(0.95, 0.8),bbox_transform=plt.gcf().transFigure)
ax.text(.66, .85, 'Finesse = 14300', color='black', fontsize = 18 , bbox=dict(facecolor='none', edgecolor='black'), horizontalalignment='left', transform=ax.transAxes)
plt.tight_layout()

folder = r'D:\measuring\data\20161025'
plt.savefig(os.path.join(folder,'LT_OFFD_Finesse.eps'))

In [ ]:
c = 3.e8
filepath = 'D:\measuring\data/20161025/linewidth_data_LT.txt'
d = np.loadtxt(filepath)
lw,u_lw = d[:,0], d[:,1]
lw_1 = lw[0:2]
lw_2 = lw[2:5]
u_lw_1 = u_lw[0:2]
u_lw_2 = u_lw[2:5]

filepath2 = 'D:\measuring\data/20161025/cavity_length_data_LT.txt'
t = np.loadtxt(filepath2)
L, u_L = t[:,0]*1.e6, t[:,1]*1.e6
L_1 = L[0:2]
L_2 = L[2:5]
u_L_1 = u_L[0:2]
u_L_2 = u_L[2:5]

print L, u_L, lw, u_lw
print L_1
fig,ax = plt.subplots(figsize=(6,5))
eb = plt.errorbar(L_1, lw_1, yerr=u_lw_1, xerr=u_L_1, fmt='o', color='b', label = 'Pos 1')
eb = plt.errorbar(L_2, lw_2, yerr=u_lw_2, xerr=u_L_2, fmt='o', color='g', label = 'Pos 2')
ax.set_xlabel("Cavity length (um)", fontsize = 18)
ax.set_ylabel("Linewidth (GHz)", fontsize = 18)
ax.legend(numpoints=1)
ax.set_xlim(4,12)
#ax.set_xlim(4,15)
folder = r'D:\measuring\data\20161025'
plt.savefig(os.path.join(folder,'LT_OND_Finesse.eps'))
